In [6]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

url = 'https://unstats.un.org/sdgs/metadata/'
response = requests.get(url)
html_content = response.content

tree = etree.HTML(html_content)
# soup = BeautifulSoup(response.text, 'html.parser')
# data = response.json()


In [8]:
downloaded_path = r'../un_indicator_metadata'
web_root = r'https://unstats.un.org'


downloaded_xpath = '/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[1]/div[2]/ul[1]/li/ul/li[1]/small/span/a[2]/@href'
downloaded_url = f'{web_root}{tree.xpath(downloaded_xpath)[0]}'
data = requests.get(downloaded_url)
if data.status_code == 200:
    with open('test.docx', 'wb') as f:
      for chunk in data.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [ ]:
from docx import Document

doc = Document('test.docx')
# paragraphs = []
# for paragraph in doc.paragraphs:
#     paragraphs.append(paragraph.text)
print(doc)

In [ ]:
goal_num = 0
collection = dict()
downloaded_path = r'../un_indicator_metadata/'

for _ in range(1, 18):
    goal_num+=1
    target_num = 0
    while True:
        target_num+=1
        indicator_num = 0
        while True:
            indicator_num+=1
            
            # Scrape words
            num_xpath = f'/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[{goal_num}]/div[2]/ul[{target_num}]/li[{indicator_num}]/small/strong/text()[normalize-space()]'
            text_xpath = f'/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[{goal_num}]/div[2]/ul[{target_num}]/li[{indicator_num}]/small/text()[normalize-space()]'
            indicator_code = tree.xpath(num_xpath)
            indicator_text = tree.xpath(text_xpath)

            if not indicator_code or not indicator_text:
                break
            indicator_code_res = indicator_code[0].strip().replace(':','').replace(' ', '').replace('.','-')
            indicator_text_res = indicator_text[0].strip()

            # Download doc files
            # Only one indicator
            downloaded_xpath_3 = f'/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[{goal_num}]/div[2]/ul[{target_num}]/li/ul/li[1]/small/span/a[2]/@href'
            # more than one indicator
            downloaded_xpath_2 = f'/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[{goal_num}]/div[2]/ul[{target_num}]/li[{indicator_num}]/ul/li/small/span/a[2]/@href'
            # more than one files
            downloaded_xpath_1 = f'/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[{goal_num}]/div[2]/ul[{target_num}]/li[{indicator_num}]/ul/small/span/a[2]/@href'
            
            if tree.xpath(downloaded_xpath_1):
                downloaded_xpath_part = tree.xpath(downloaded_xpath_1)
            elif tree.xpath(downloaded_xpath_2):
                downloaded_xpath_part = tree.xpath(downloaded_xpath_2)
            elif tree.xpath(downloaded_xpath_3):
                downloaded_xpath_part = tree.xpath(downloaded_xpath_3)
            else:
                print(f"{indicator_code_res}, No files")
                break
            
            # if downloaded_xpath_part:
            downloaded_url = f'{web_root}{downloaded_xpath_part[0]}'
            data = requests.get(downloaded_url)
            print(fr'{downloaded_path}{indicator_code_res}.docx')
            if data.status_code == 200:
                with open(fr'{downloaded_path}{indicator_code_res}.docx', 'wb') as f:
                    for chunk in data.iter_content(1024 * 1024 * 2):  # 2 MB chunks
                        f.write(chunk)

            collection[indicator_code_res] = indicator_text_res
        
        next_target_num = target_num + 1
        next_target_xpath = f'/html/body/div[1]/div/section/div/div/div/div/div[2]/div[2]/div[{goal_num}]/div[2]/ul[{next_target_num}]'
        next_target_exists = tree.xpath(next_target_xpath)
        
        if not next_target_exists:
            break

In [32]:
import pandas as pd

indicators = pd.DataFrame(pd.Series(collection, name='Indicator')).reset_index(names='ID')

In [33]:
from docx import Document
import os
import re
folder =  r'../un_indicator_metadata/'
filenames = os.listdir(folder)
filename_list = [filename for filename in filenames if filename.endswith('.docx')]
# document = Document(filename)
# __full__text = []

def read_docx_paragraphs(file_path):
    doc = Document(file_path)
    paragraphs = []
    for paragraph in doc.paragraphs:
        paragraphs.append(paragraph.text)
    return paragraphs

def find_definition_and_concepts(paragraphs):
    found = False
    definition_and_concepts = []
    for paragraph in paragraphs:
        if found:
            if paragraph.startswith("2.b. Unit of measure"):
                break
            definition_and_concepts.append(paragraph)
        if paragraph.startswith("2. Definition, concepts, and classifications"):
            found = True
    return definition_and_concepts

# paragraphs = read_docx_paragraphs(file_path=filename)
# target_paragraph = find_definition_and_concepts(paragraphs)
# print(target_paragraph)
# definitions = list()
for idx, filename in enumerate(filename_list):
    # match = re.search(r'-(\d{2}-\w{2}-\d{2})\.docx', filename)
    # matched_text = re.sub(r'0','',match.group(1))
    # matched_text = matched_text.replace('-','.')
    paragraphs = read_docx_paragraphs(file_path=os.path.join(folder, filename))
    target_paragraph = find_definition_and_concepts(paragraphs=paragraphs)
    definition = ''
    for paragraph in target_paragraph[1:]:
        definition+=paragraph
    # definitions.append(definition)
    
    indicators.loc[(indicators['ID']==filename[:-5]), 'Description'] = definition
    

In [41]:
# Validation
pd.set_option('display.max_colwidth', None)
indicators.loc[indicators['ID']=='Indicator1-4-1', 'Description']

4    Definition:The proportion of population living in households with access to basic services is defined as the proportion of population using public service provision systems that meet basic human needs including drinking water, sanitation, hygiene, energy, mobility, waste collection, health care, education and information technologies. The basic services indicator is therefore based on 9 components. These components are captured in various standalone indicators of the SDGs, which means that the concepts and definitions of SDG indicator 1.4.1 will be derived from or are the same as those of these specific SDG indicators. Concepts:The term ‘access to basic services’ implies that sufficient and affordable service is reliably available with adequate quality.Access to Basic Drinking Water Services refers to the use of drinking water from an improved source with a collection time of not more than 30 minutes for a round trip, including queuing. ‘Improved’ drinking water sources include th

In [43]:
indicators.to_csv('un_sdgs_indicators.csv')